# Tutorial 14: Chaotic Systems

Exploring chaos, Lyapunov exponents, and sensitive dependence.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mechanics_dsl import PhysicsCompiler

compiler = PhysicsCompiler()

def simulate_double_pendulum(theta1_0):
    dsl_code = rf"""
    \system{{double_pendulum}}
    \defvar{{theta1}}{{Angle 1}}{{rad}}
    \defvar{{theta2}}{{Angle 2}}{{rad}}
    \parameter{{m1}}{{1.0}}{{kg}}
    \parameter{{m2}}{{1.0}}{{kg}}
    \parameter{{L1}}{{1.0}}{{m}}
    \parameter{{L2}}{{1.0}}{{m}}
    \parameter{{g}}{{9.81}}{{m/s^2}}
    \lagrangian{{
        \frac{{1}}{{2}}*(m1+m2)*L1^2*\dot{{theta1}}^2 +
        \frac{{1}}{{2}}*m2*L2^2*\dot{{theta2}}^2 +
        m2*L1*L2*\dot{{theta1}}*\dot{{theta2}}*\cos{{theta1-theta2}} +
        (m1+m2)*g*L1*\cos{{theta1}} + m2*g*L2*\cos{{theta2}}
    }}
    \initial{{theta1={theta1_0}, theta2=2.0, theta1_dot=0.0, theta2_dot=0.0}}
    """
    compiler.compile_dsl(dsl_code)
    return compiler.simulate(t_span=(0, 15), num_points=1500)

In [ ]:
# Simulate two nearly identical systems
sol1 = simulate_double_pendulum(2.0)
sol2 = simulate_double_pendulum(2.001)  # Tiny difference!

t = sol1['t']
theta1_a, theta1_b = sol1['y'][0], sol2['y'][0]

# Calculate divergence
divergence = np.abs(theta1_a - theta1_b)

fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(t, theta1_a, 'b-', lw=1, label='θ₁ = 2.000')
axes[0].plot(t, theta1_b, 'r--', lw=1, label='θ₁ = 2.001')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('θ₁ (rad)')
axes[0].set_title('Sensitive Dependence on Initial Conditions')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].semilogy(t, divergence + 1e-10, 'purple', lw=2)
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('|Δθ₁| (rad)')
axes[1].set_title('Exponential Divergence (Chaos)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()